In [99]:
import numpy as np
import pandas as pd
import matplotlib as plt
from torch import torch, nn
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

In [100]:
torch.cuda.is_available()

True

In [101]:
torch.cuda.get_device_name()

'NVIDIA GeForce GTX 1080'

In [102]:
device = "cuda" if torch.cuda.is_available() else "cpu"
device

'cuda'

In [ ]:
# Load the heart disease dataset:
columns = ['Age', 'Oldpeak', 'MaxHR', 'HeartDisease']
heart_disease_dataframe = pd.read_csv("../../Data/heart.csv")[columns]

# Select out X and Y
X = heart_disease_dataframe[['Age', 'Oldpeak', 'MaxHR']].values
y = heart_disease_dataframe['HeartDisease'].values

# Apply standard scaler to X.
X = StandardScaler().fit_transform(X)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.30, random_state=42)

X_train_tensor = torch.Tensor(torch.from_numpy(X_train.astype(np.float32)))
y_train_tensor = torch.Tensor(torch.from_numpy(y_train.astype(np.float32))).reshape(-1,1)
X_test_tensor = torch.Tensor(torch.from_numpy(X_test.astype(np.float32)))
y_test_tensor = torch.Tensor(torch.from_numpy(y_test.astype(np.float32))).reshape(-1,1)

In [114]:
learning_rate = 0.01
num_classes = 10
class SimpleNeuralNet(nn.Module):
    def __init__(self):
        super().__init__()
        self.linear = torch.nn.Linear(28**2, 20)
        self.linear2 = torch.nn.Linear(20, num_classes)


    def forward(self, x):
        x = x.view(-1, 28**2)
        z1 = self.linear(x)
        x2 = F.relu(z1)
        z2 = self.linear2(x2)
        return F.log_softmax(z2, dim=1) # Warning if no dimensions for some reason

nn_model = SimpleNeuralNet().cuda()
nn_model_optimizer = optim.SGD(nn_model.parameters(), lr=learning_rate)
summary(nn_model, (1, 28, 28))

TypeError: 'int' object is not subscriptable